In [8]:
import pandas as pd
import numpy as np

# Baca Model
    

# Baca Dataset
def readDataset():
    df = pd.read_csv('iris.csv')
    return df

# Fungsi Aktivasi
def linearActivation(x):
    return x

def sigmoidActivation(x):
    value = float(1 / (1 + math.exp(x * -1)))
    return value
    
def reluActivation(x):
    return max(0, x)

def softmaxActivation(x):
    x = np.exp(x)
    return x / x.sum()

# Fungsi Loss Sum Squared Error untuk Linear, Sigmoid, and Relu activation
def lossSumSquareddError(x, t, o, outputs): 
    sumE = 0
    for i in outputs:
        sumE += (t[i] - o[i])**2
    return 1/2 * sumE

# Fungsi Loss untuk softmax
def lossCrossEntropy(pk):
    return -1*math.log(pk)

# Fungsi Turunan Relu
def turunanRelu(x):
    return 0 if x < 0 else 1
    
# Fungsi Turunan Sigmoid
def turunanSigmoid(x):
    return sigmoidActivation(x) * (1-sigmoidActivation(x))

# Fungsi Turunan Softmax
def turunanSoftmax(x, targetClass):
    return (-(1-x)) if targetClass else x

# Chain Rule

# Chain Rule for 

# Neural Network
# Class Neuron (?)
# Class Layer

# Main program
